<a href="https://colab.research.google.com/github/twelvesense/first-repository/blob/master/ExplNode07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 프로젝트: 뉴스기사 요약해보기
## 새로운 데이터셋에 대해서 추상적 요약과 추출적 요약을 모두 해보는 시간을 가져봐요.

In [ ]:
# 먼저 주요 라이브러리 버전을 확인해 보죠.
from importlib.metadata import version
import nltk
import tensorflow
import summa
import pandas as pd
import urllib.request

print(nltk.__version__)
print(tensorflow.__version__)
print(version('summa'))

3.7
2.9.0
1.2.0


# Step 1. 데이터 수집하기

In [ ]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/sunnysai12345/News_Summary/master/news_summary_more.csv", filename="news_summary_more.csv")
data = pd.read_csv('news_summary_more.csv', encoding='iso-8859-1')

In [ ]:
# data.sample(10)
data.sample(10)

,headlines,text
70636,People gather to shout Bolna Aunty Aun Kya in ...,People in Delhi and Mumbai gathered to sing th...
16725,RBI gives Yes Bank's Rana Kapoor only 4 more m...,The RBI has asked Yes Bank's billionaire Manag...
86354,"Phogat demands stopping Ind-Pak match, cheers ...",Wrestler Babita Phogat cheered for India on Tw...
76569,China denies hiring foreigners to run its radi...,China has denied reports that it is planning t...
65152,WWE star Triple H attempts Amitabh's 'Shahensh...,"Former WWE champion Triple H, in an interview ..."
39074,He was my protector: Celeb chef Anthony's girl...,"Celebrity chef Anthony Bourdain's girlfriend, ..."
29799,Surgeon builds windpipes from largest artery i...,French doctors led by surgeon Emmanuel Martino...
43165,US won't succeed if it tries to divide EU on t...,German economy minister Peter Altmaier has sai...
51291,Which industry in India pays the most taxes?,India's steel sector is taxed the highest amon...
2086,Nadal wakes up sleeping journalist during his ...,World number two Rafael Nadal woke up a sleepi...


추상적 요약을 하는 경우에는 text를 본문, headlines를 이미 요약된 데이터로 삼아서 모델을 학습할 수 있어요.<br>
추출적 요약을 하는 경우에는 오직 text열만을 사용하세요.

# Step 2. 데이터 전처리하기 (추상적 요약)
실습에서 사용된 전처리를 참고하여 각자 필요하다고 생각하는 전처리를 추가 사용하여 텍스트를 정규화 또는 정제해 보세요. 만약, 불용어 제거를 선택한다면 상대적으로 길이가 짧은 요약 데이터에 대해서도 불용어를 제거하는 것이 좋을지 고민해 보세요.

# Step 3. 어텐션 메커니즘 사용하기 (추상적 요약)
일반적인 seq2seq보다는 어텐션 메커니즘을 사용한 seq2seq를 사용하는 것이 더 나은 성능을 얻을 수 있어요. 실습 내용을 참고하여 어텐션 메커니즘을 사용한 seq2seq를 설계해 보세요.

# Step 4. 실제 결과와 요약문 비교하기 (추상적 요약)
원래의 요약문(headlines 열)과 학습을 통해 얻은 추상적 요약의 결과를 비교해 보세요.

# Step 5. Summa을 이용해서 추출적 요약해보기
추상적 요약은 추출적 요약과는 달리 문장의 표현력을 다양하게 가져갈 수 있지만, 추출적 요약에 비해서 난이도가 높아요. 반대로 말하면 추출적 요약은 추상적 요약에 비해 난이도가 낮고 기존 문장에서 문장을 꺼내오는 것이므로 잘못된 요약이 나올 가능성이 낮아요.
Summa의 summarize를 사용하여 추출적 요약을 해보세요.